# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [94]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [95]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yerbogachen,61.2767,108.0108,86.94,34.0,4.0,8.99,RU,1.627360e+09
1,salalah,17.0151,54.0924,78.89,89.0,75.0,5.75,OM,1.627360e+09
2,baykit,61.6700,96.3700,86.88,30.0,2.0,11.07,RU,1.627360e+09
3,hermanus,-34.4187,19.2345,54.90,68.0,1.0,11.18,ZA,1.627360e+09
4,east london,-33.0153,27.9116,63.54,29.0,0.0,5.75,ZA,1.627359e+09
...,...,...,...,...,...,...,...,...,...
607,palimbang,6.2106,124.1894,85.71,69.0,92.0,7.11,PH,1.627360e+09
608,salta,-24.7859,-65.4117,52.09,69.0,90.0,1.01,AR,1.627360e+09
609,payson,40.0444,-111.7321,81.01,53.0,1.0,1.01,US,1.627360e+09
610,nara,34.6851,135.8049,92.44,50.0,97.0,8.01,JP,1.627360e+09


In [96]:
cities = cities.dropna()
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yerbogachen,61.2767,108.0108,86.94,34.0,4.0,8.99,RU,1.627360e+09
1,salalah,17.0151,54.0924,78.89,89.0,75.0,5.75,OM,1.627360e+09
2,baykit,61.6700,96.3700,86.88,30.0,2.0,11.07,RU,1.627360e+09
3,hermanus,-34.4187,19.2345,54.90,68.0,1.0,11.18,ZA,1.627360e+09
4,east london,-33.0153,27.9116,63.54,29.0,0.0,5.75,ZA,1.627359e+09
...,...,...,...,...,...,...,...,...,...
607,palimbang,6.2106,124.1894,85.71,69.0,92.0,7.11,PH,1.627360e+09
608,salta,-24.7859,-65.4117,52.09,69.0,90.0,1.01,AR,1.627360e+09
609,payson,40.0444,-111.7321,81.01,53.0,1.0,1.01,US,1.627360e+09
610,nara,34.6851,135.8049,92.44,50.0,97.0,8.01,JP,1.627360e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [97]:
humidity = cities["Humidity"]
max_humidity = humidity.max()

In [98]:
locations = cities[["Lat","Lng"]]

###  Please note the Humidity Heat Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [99]:
# Plot Heatmap
fig = gmaps.figure()

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [100]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [101]:
ideal_cities = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) 
                          & (cities["Wind Speed"] < 10) & (cities["Cloudiness"] == 0)]
ideal_cities.head()                                                                                                                

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,byron bay,-28.6500,153.6167,73.51,43.0,0.0,7.36,AU,1.627360e+09
72,virden,39.5009,-89.7679,73.71,89.0,0.0,4.94,US,1.627360e+09
106,jalu,29.0331,21.5482,77.83,41.0,0.0,6.85,LY,1.627360e+09
121,esfarayen,37.0765,57.5101,76.66,34.0,0.0,7.72,IR,1.627360e+09
192,pallasovka,50.0500,46.8833,72.25,38.0,0.0,8.55,RU,1.627360e+09


In [102]:
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,byron bay,-28.6500,153.6167,73.51,43.0,0.0,7.36,AU,1.627360e+09
72,virden,39.5009,-89.7679,73.71,89.0,0.0,4.94,US,1.627360e+09
106,jalu,29.0331,21.5482,77.83,41.0,0.0,6.85,LY,1.627360e+09
121,esfarayen,37.0765,57.5101,76.66,34.0,0.0,7.72,IR,1.627360e+09
192,pallasovka,50.0500,46.8833,72.25,38.0,0.0,8.55,RU,1.627360e+09


In [103]:
ideal_cities.reset_index(inplace=True)
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,58,byron bay,-28.6500,153.6167,73.51,43.0,0.0,7.36,AU,1.627360e+09
1,72,virden,39.5009,-89.7679,73.71,89.0,0.0,4.94,US,1.627360e+09
2,106,jalu,29.0331,21.5482,77.83,41.0,0.0,6.85,LY,1.627360e+09
3,121,esfarayen,37.0765,57.5101,76.66,34.0,0.0,7.72,IR,1.627360e+09
4,192,pallasovka,50.0500,46.8833,72.25,38.0,0.0,8.55,RU,1.627360e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
hotel_list = []

In [105]:
len(ideal_cities)

16

In [106]:
for x in range(len(ideal_cities)):
    lat = ideal_cities.loc[x]["Lat"]
    lng = ideal_cities.loc[x]["Lng"]
    
    params = {
            "location": f'{lat},{lng}',
            "radius": 5000,
            "types": "hotel",
            "key": g_key
            }
        
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    hotel_jsn = requested.json()
    try:
          hotel_list.append(hotel_jsn['results'][0]['name'])
    except:
          hotel_list.append("")
                
ideal_cities["Hotel Name"] = hotel_list
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,58,byron bay,-28.6500,153.6167,73.51,43.0,0.0,7.36,AU,1.627360e+09,Byron Bay
1,72,virden,39.5009,-89.7679,73.71,89.0,0.0,4.94,US,1.627360e+09,Virden
2,106,jalu,29.0331,21.5482,77.83,41.0,0.0,6.85,LY,1.627360e+09,Jalu
3,121,esfarayen,37.0765,57.5101,76.66,34.0,0.0,7.72,IR,1.627360e+09,Esfarāyen
4,192,pallasovka,50.0500,46.8833,72.25,38.0,0.0,8.55,RU,1.627360e+09,Pallasovka


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]


###  Please note the Hotel Map screenshot was placed in the Images folder \Python-API-challenge\Images

In [108]:
# Add marker layer ontop of heat map
# markers = gmaps.marker_layer(locations)
locations = ideal_cities[["Lat", "Lng"]]
fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

#fig = gmaps.figure()
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))